In [2]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
from math import log
from decimal import Decimal
import altair as alt
import kagglehub as kh

In [3]:
# nba endpoints

## get miami heat team_id
from nba_api.stats.static import teams

## get players
from nba_api.stats.static import players

from nba_api.stats.endpoints import BoxScoreTraditionalV2

from nba_api.stats.endpoints import PlayerIndex

from nba_api.stats.endpoints import playercareerstats

## Offensive Stat Predictions
idea- pick an offensive player and his most likely defensive matchup on opposing team, to predict possible game scores
- need to get position of player predicting on
- get shot diets
- get position defender most likely to guard
- get strength of schedule of team

### Get all Heat players, check if they are there (changing to just Herro)

In [ ]:
all_players=PlayerIndex().get_data_frames()[0]
all_players

,PERSON_ID,PLAYER_LAST_NAME,PLAYER_FIRST_NAME,PLAYER_SLUG,TEAM_ID,TEAM_SLUG,IS_DEFUNCT,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,...,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,ROSTER_STATUS,FROM_YEAR,TO_YEAR,PTS,REB,AST,STATS_TIMEFRAME
0,1630173,Achiuwa,Precious,precious-achiuwa,1610612752,knicks,0,New York,Knicks,NYK,...,2020.0,1.0,20.0,1.0,2020,2024,6.3,5.6,1.0,Season
1,203500,Adams,Steven,steven-adams,1610612745,rockets,0,Houston,Rockets,HOU,...,2013.0,1.0,12.0,1.0,2013,2024,3.5,5.3,1.2,Season
2,1628389,Adebayo,Bam,bam-adebayo,1610612748,heat,0,Miami,Heat,MIA,...,2017.0,1.0,14.0,1.0,2017,2024,17.6,9.8,4.4,Season
3,1630534,Agbaji,Ochai,ochai-agbaji,1610612761,raptors,0,Toronto,Raptors,TOR,...,2022.0,1.0,14.0,1.0,2022,2024,10.1,3.7,1.5,Season
4,1630583,Aldama,Santi,santi-aldama,1610612763,grizzlies,0,Memphis,Grizzlies,MEM,...,2021.0,1.0,30.0,1.0,2021,2024,12.8,6.6,2.9,Season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,1629027,Young,Trae,trae-young,1610612737,hawks,0,Atlanta,Hawks,ATL,...,2018.0,1.0,5.0,1.0,2018,2024,24.1,3.2,11.4,Season
565,1627826,Zubac,Ivica,ivica-zubac,1610612746,clippers,0,LA,Clippers,LAC,...,2016.0,2.0,32.0,1.0,2016,2024,16.4,12.5,2.5,Season
566,1641783,da Silva,Tristan,tristan-da-silva,1610612753,magic,0,Orlando,Magic,ORL,...,2024.0,1.0,18.0,1.0,2024,2024,7.4,3.4,1.5,Season
567,1628427,Čančar,Vlatko,vlatko-čančar,1610612743,nuggets,0,Denver,Nuggets,DEN,...,2017.0,2.0,49.0,1.0,2019,2024,1.6,2.6,0.7,Season


In [40]:
# get dyson daniels and max strus from all_players as defensive tests
dyson = all_players[(all_players['PLAYER_FIRST_NAME']=='Dyson') & (all_players['PLAYER_LAST_NAME']=='Daniels')]
strus = all_players[(all_players['PLAYER_FIRST_NAME']=='Max') & (all_players['PLAYER_LAST_NAME']=='Strus')]


In [41]:
dyson

,PERSON_ID,PLAYER_LAST_NAME,PLAYER_FIRST_NAME,PLAYER_SLUG,TEAM_ID,TEAM_SLUG,IS_DEFUNCT,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,...,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,ROSTER_STATUS,FROM_YEAR,TO_YEAR,PTS,REB,AST,STATS_TIMEFRAME
112,1630700,Daniels,Dyson,dyson-daniels,1610612737,hawks,0,Atlanta,Hawks,ATL,...,2022.0,1.0,8.0,1.0,2022,2024,14.0,5.7,4.3,Season


In [47]:
strus

,PERSON_ID,PLAYER_LAST_NAME,PLAYER_FIRST_NAME,PLAYER_SLUG,TEAM_ID,TEAM_SLUG,IS_DEFUNCT,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,...,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,ROSTER_STATUS,FROM_YEAR,TO_YEAR,PTS,REB,AST,STATS_TIMEFRAME
477,1629622,Strus,Max,max-strus,1610612739,cavaliers,0,Cleveland,Cavaliers,CLE,...,NaN,NaN,NaN,1.0,2019,2024,9.5,4.3,3.0,Season


In [26]:
# get all players for miami heat (herro only)
herro = all_players[all_players['PLAYER_LAST_NAME']=='Herro']
herro

,PERSON_ID,PLAYER_LAST_NAME,PLAYER_FIRST_NAME,PLAYER_SLUG,TEAM_ID,TEAM_SLUG,IS_DEFUNCT,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,...,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,ROSTER_STATUS,FROM_YEAR,TO_YEAR,PTS,REB,AST,STATS_TIMEFRAME
205,1629639,Herro,Tyler,tyler-herro,1610612748,heat,0,Miami,Heat,MIA,...,2019.0,1.0,13.0,1.0,2019,2024,23.4,5.2,5.6,Season


In [27]:
 # keep only certain columns
herro = herro[['PERSON_ID','PLAYER_FIRST_NAME','PLAYER_LAST_NAME','POSITION']].reset_index(drop=True)
herro

,PERSON_ID,PLAYER_FIRST_NAME,PLAYER_LAST_NAME,POSITION
0,1629639,Tyler,Herro,G


In [31]:
# Create empty list to store career stats
season_stats = []

# Loop through each player ID and get their career stats
for player_id in herro['PERSON_ID']:
    try:
        # Get career stats for player
        player_stats = playercareerstats.PlayerCareerStats(player_id=player_id)
        # Get the data frame from the response
        stats_df = player_stats.get_data_frames()[0]
        # Append to list
        season_stats.append(stats_df)
    except:
        print(f"Could not get stats for player ID {player_id}")
        continue

# Combine all stats into single dataframe if list is not empty
if season_stats:
    herro_all_season_stats = pd.concat(season_stats)
else:
    print("No career stats were retrieved")



In [32]:
herro_all_season_stats

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1629639,2019-20,00,1610612748,MIA,20.0,55,8,1508.0,270,...,0.870,16,208,224,121,34,9,87,76,743
1,1629639,2020-21,00,1610612748,MIA,21.0,54,15,1635.0,307,...,0.803,25,243,268,184,35,17,101,71,815
2,1629639,2021-22,00,1610612748,MIA,22.0,66,10,2151.0,501,...,0.868,32,297,329,263,44,8,174,95,1367
3,1629639,2022-23,00,1610612748,MIA,23.0,67,67,2335.0,487,...,0.934,27,333,360,280,51,16,158,101,1347
4,1629639,2023-24,00,1610612748,MIA,24.0,42,40,1407.0,324,...,0.856,19,203,222,188,30,3,93,63,875
5,1629639,2024-25,00,1610612748,MIA,25.0,67,67,2381.0,553,...,0.864,26,325,351,375,56,16,174,79,1566


In [33]:
# filter for season_id = 2024-25
herro_season_total_2425 = herro_all_season_stats[herro_all_season_stats['SEASON_ID'] == "2024-25"]
herro_season_total_2425


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
5,1629639,2024-25,00,1610612748,MIA,25.0,67,67,2381.0,553,...,0.864,26,325,351,375,56,16,174,79,1566


In [34]:
# join with the miami_heat_players dataframe to get names
herro_season_total_2425 = herro.merge(herro_season_total_2425, left_on='PERSON_ID', right_on='PLAYER_ID', how='right')
herro_season_total_2425

,PERSON_ID,PLAYER_FIRST_NAME,PLAYER_LAST_NAME,POSITION,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1629639,Tyler,Herro,G,1629639,2024-25,00,1610612748,MIA,25.0,...,0.864,26,325,351,375,56,16,174,79,1566


In [35]:
# remove league_id, teame_id,player age, gs
herro_season_total_2425 = herro_season_total_2425.drop(columns=['LEAGUE_ID', 'TEAM_ID', 'PLAYER_AGE', 'GS'])
herro_season_total_2425

,PERSON_ID,PLAYER_FIRST_NAME,PLAYER_LAST_NAME,POSITION,PLAYER_ID,SEASON_ID,TEAM_ABBREVIATION,GP,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1629639,Tyler,Herro,G,1629639,2024-25,MIA,67,2381.0,553,...,0.864,26,325,351,375,56,16,174,79,1566


In [36]:
# calculate season averages for players by dividing stats by GP
herro_season_total_2425['PPG'] = herro_season_total_2425['PTS'] / herro_season_total_2425['GP']
herro_season_total_2425['RPG'] = herro_season_total_2425['REB'] / herro_season_total_2425['GP']
herro_season_total_2425['APG'] = herro_season_total_2425['AST'] / herro_season_total_2425['GP']
herro_season_total_2425['SPG'] = herro_season_total_2425['STL'] / herro_season_total_2425['GP']
herro_season_total_2425['BPG'] = herro_season_total_2425['BLK'] / herro_season_total_2425['GP']
herro_season_total_2425['TOVPG'] = herro_season_total_2425['TOV'] / herro_season_total_2425['GP']

In [38]:
herro_season_avg_2425 = herro_season_total_2425.round(1)

### Get Player Defensive Stats

In [44]:
from nba_api.stats.endpoints import LeagueHustleStatsPlayer
player_d_stats_per_game = LeagueHustleStatsPlayer(season='2024-25',per_mode_time='PerGame').get_data_frames()[0]
player_d_stats_per_game


,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,G,MIN,CONTESTED_SHOTS,CONTESTED_SHOTS_2PT,CONTESTED_SHOTS_3PT,...,PCT_LOOSE_BALLS_RECOVERED_DEF,OFF_BOXOUTS,DEF_BOXOUTS,BOX_OUTS,BOX_OUT_PLAYER_TEAM_REBS,BOX_OUT_PLAYER_REBS,PCT_BOX_OUTS_OFF,PCT_BOX_OUTS_DEF,PCT_BOX_OUTS_TEAM_REB,PCT_BOX_OUTS_REB
0,1630639,A.J. Lawson,1610612761,TOR,24.0,13,21.1,2.00,1.08,0.92,...,0.750,0.00,0.31,0.31,0.31,0.15,0.000,1.000,1.000,0.500
1,1631260,AJ Green,1610612749,MIL,25.0,61,22.7,3.25,1.90,1.34,...,0.630,0.03,0.49,0.52,0.49,0.11,0.063,0.938,0.938,0.219
2,1642358,AJ Johnson,1610612764,WAS,20.0,10,20.7,3.30,1.90,1.40,...,0.750,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.000
3,203932,Aaron Gordon,1610612743,DEN,29.0,42,27.3,3.12,2.19,0.93,...,0.417,0.12,0.31,0.43,0.43,0.38,0.278,0.722,1.000,0.889
4,1628988,Aaron Holiday,1610612745,HOU,28.0,49,13.4,2.10,1.06,1.04,...,0.308,0.00,0.29,0.29,0.27,0.06,0.000,1.000,0.929,0.214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,203897,Zach LaVine,1610612758,SAC,30.0,61,34.8,4.72,1.57,3.15,...,0.625,0.03,0.10,0.13,0.13,0.08,0.250,0.750,1.000,0.625
547,1630192,Zeke Nnaji,1610612743,DEN,24.0,43,11.7,3.79,2.44,1.35,...,0.500,0.05,0.33,0.37,0.37,0.21,0.125,0.875,1.000,0.563
548,1630533,Ziaire Williams,1610612751,BKN,23.0,54,25.1,3.67,1.69,1.98,...,0.348,0.07,0.31,0.39,0.39,0.20,0.190,0.810,1.000,0.524
549,1629627,Zion Williamson,1610612740,NOP,24.0,30,28.6,4.17,1.70,2.47,...,0.323,0.17,0.43,0.60,0.60,0.43,0.278,0.722,1.000,0.722


In [45]:
# filter for dyson daniels
dyson_d_stats_per_game = player_d_stats_per_game[player_d_stats_per_game['PLAYER_ID'] == 1630700]
dyson_d_stats_per_game


,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,G,MIN,CONTESTED_SHOTS,CONTESTED_SHOTS_2PT,CONTESTED_SHOTS_3PT,...,PCT_LOOSE_BALLS_RECOVERED_DEF,OFF_BOXOUTS,DEF_BOXOUTS,BOX_OUTS,BOX_OUT_PLAYER_TEAM_REBS,BOX_OUT_PLAYER_REBS,PCT_BOX_OUTS_OFF,PCT_BOX_OUTS_DEF,PCT_BOX_OUTS_TEAM_REB,PCT_BOX_OUTS_REB
158,1630700,Dyson Daniels,1610612737,ATL,22.0,65,33.9,6.02,3.17,2.85,...,0.609,0.06,0.26,0.34,0.31,0.15,0.182,0.773,0.952,0.476


In [48]:
# max strus defensive stats
strus_d_stats_per_game = player_d_stats_per_game[player_d_stats_per_game['PLAYER_ID'] == 1629622]
strus_d_stats_per_game


,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,G,MIN,CONTESTED_SHOTS,CONTESTED_SHOTS_2PT,CONTESTED_SHOTS_3PT,...,PCT_LOOSE_BALLS_RECOVERED_DEF,OFF_BOXOUTS,DEF_BOXOUTS,BOX_OUTS,BOX_OUT_PLAYER_TEAM_REBS,BOX_OUT_PLAYER_REBS,PCT_BOX_OUTS_OFF,PCT_BOX_OUTS_DEF,PCT_BOX_OUTS_TEAM_REB,PCT_BOX_OUTS_REB
383,1629622,Max Strus,1610612739,CLE,28.0,41,25.1,3.17,1.8,1.37,...,0.423,0.02,0.41,0.44,0.41,0.27,0.056,0.944,0.944,0.611


### Positional Matchup Data?

In [25]:
from nba_api.stats.endpoints import matchupsrollup
herro_matchups_rollup = matchupsrollup.MatchupsRollup(season='2024-25',off_player_id_nullable=1629639).get_data_frames()[0]
herro_matchups_rollup


,SEASON_ID,POSITION,PERCENT_OF_TIME,OFF_PLAYER_ID,OFF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,TEAM_PTS,...,MATCHUP_BLK,MATCHUP_FGM,MATCHUP_FGA,MATCHUP_FG_PCT,MATCHUP_FG3M,MATCHUP_FG3A,MATCHUP_FG3_PCT,MATCHUP_FTM,MATCHUP_FTA,SFL
0,22024,F,0.243964,1629639,Tyler Herro,66,231:60,1199.1,449,1429,...,11,171,381,0.449,64,182,0.352,43,46,19
1,22024,C,0.052918,1629639,Tyler Herro,60,50:19,267.2,175,412,...,8,67,143,0.469,26,64,0.406,15,16,5
2,22024,G,0.703118,1629639,Tyler Herro,67,668:37,3332.4,1056,3740,...,29,394,846,0.466,147,403,0.365,121,147,70
3,22024,TOTAL,1.000000,1629639,Tyler Herro,67,950:56,4798.7,1680,5581,...,48,632,1370,0.461,237,649,0.365,179,209,94


In [49]:
# dyson daniels matchups
dyson_matchups_rollup = matchupsrollup.MatchupsRollup(season='2024-25',off_player_id_nullable=1630700).get_data_frames()[0]
dyson_matchups_rollup


,SEASON_ID,POSITION,PERCENT_OF_TIME,OFF_PLAYER_ID,OFF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,TEAM_PTS,...,MATCHUP_BLK,MATCHUP_FGM,MATCHUP_FGA,MATCHUP_FG_PCT,MATCHUP_FG3M,MATCHUP_FG3A,MATCHUP_FG3_PCT,MATCHUP_FTM,MATCHUP_FTA,SFL
0,22024,G,0.685270,1630700,Dyson Daniels,65,559:29,3094.8,551,3460,...,13,237,498,0.476,37,111,0.333,40,68,36
1,22024,F,0.244902,1630700,Dyson Daniels,65,199:57,1139.7,241,1359,...,8,104,210,0.495,17,46,0.370,16,26,14
2,22024,C,0.069828,1630700,Dyson Daniels,57,57:01,345.5,111,415,...,5,51,109,0.468,6,20,0.300,3,4,2
3,22024,TOTAL,1.000000,1630700,Dyson Daniels,65,816:26,4580.0,903,5234,...,26,392,817,0.480,60,177,0.339,59,98,52


In [50]:
# dyson daniels matchups
dyson_matchups_rollup_def = matchupsrollup.MatchupsRollup(season='2024-25',def_player_id_nullable=1630700).get_data_frames()[0]
dyson_matchups_rollup_def


,SEASON_ID,POSITION,PERCENT_OF_TIME,DEF_PLAYER_ID,DEF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,TEAM_PTS,...,MATCHUP_BLK,MATCHUP_FGM,MATCHUP_FGA,MATCHUP_FG_PCT,MATCHUP_FG3M,MATCHUP_FG3A,MATCHUP_FG3_PCT,MATCHUP_FTM,MATCHUP_FTA,SFL
0,22024,F,0.236285,1630700,Dyson Daniels,65,213:01,1134.8,347,1487,...,6,143,266,0.538,37,98,0.378,24,35,18
1,22024,G,0.686370,1630700,Dyson Daniels,65,618:46,3164.3,719,3313,...,29,283,613,0.462,97,252,0.385,56,71,33
2,22024,C,0.077344,1630700,Dyson Daniels,57,69:44,369.9,146,590,...,4,65,109,0.596,7,20,0.350,9,11,6
3,22024,TOTAL,1.000000,1630700,Dyson Daniels,65,901:31,4669.0,1212,5390,...,39,491,988,0.497,141,370,0.381,89,117,57


### Get all Heat Games

In [21]:
nba_teams = teams.get_teams()

miami = [team for team in nba_teams if team['abbreviation'] == 'MIA'][0]
miami_id = miami['id']

In [22]:
from nba_api.stats.endpoints import leaguegamefinder

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=miami_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612748,MIA,Miami Heat,0022400948,2025-03-12,MIA vs. LAC,L,239,104,...,0.714,9,20,29,27,14,3,10,17,-15.0
1,22024,1610612748,MIA,Miami Heat,0022400931,2025-03-10,MIA vs. CHA,L,240,102,...,0.700,11,35,46,24,7,3,16,15,-3.0
2,22024,1610612748,MIA,Miami Heat,0022400916,2025-03-08,MIA vs. CHI,L,241,109,...,0.737,7,37,44,25,7,5,9,17,-5.0
3,22024,1610612748,MIA,Miami Heat,0022400907,2025-03-07,MIA vs. MIN,L,238,104,...,0.727,7,34,41,29,7,2,9,16,-2.0
4,22024,1610612748,MIA,Miami Heat,0022400893,2025-03-05,MIA @ CLE,L,239,107,...,0.880,9,33,42,29,7,2,13,19,-5.0


In [39]:
# Subset the games to when the last 4 digits of SEASON_ID were 2024
games_2024 = games[games.SEASON_ID.str[-4:] == '2024']
games_2024

games_2024_test = games_2024.head(5)
games_2024_test

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612748,MIA,Miami Heat,0022400948,2025-03-12,MIA vs. LAC,L,239,104,...,0.714,9,20,29,27,14,3,10,17,-15.0
1,22024,1610612748,MIA,Miami Heat,0022400931,2025-03-10,MIA vs. CHA,L,240,102,...,0.700,11,35,46,24,7,3,16,15,-3.0
2,22024,1610612748,MIA,Miami Heat,0022400916,2025-03-08,MIA vs. CHI,L,241,109,...,0.737,7,37,44,25,7,5,9,17,-5.0
3,22024,1610612748,MIA,Miami Heat,0022400907,2025-03-07,MIA vs. MIN,L,238,104,...,0.727,7,34,41,29,7,2,9,16,-2.0
4,22024,1610612748,MIA,Miami Heat,0022400893,2025-03-05,MIA @ CLE,L,239,107,...,0.880,9,33,42,29,7,2,13,19,-5.0


### importing common box score numbers

In [40]:
# get boxscores for all game ids in 2024
game_ids_2024_test = games_2024_test['GAME_ID'].values

# make a dictionary where game_id is the key and boxscore is value
boxscores = {}
for game_id in game_ids_2024_test:
    boxscores[game_id] = BoxScoreTraditionalV2(game_id=game_id).get_data_frames()[0]



In [42]:
boxscores['0022400948']

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0022400948,1610612746,LAC,LA,203992,Bogdan Bogdanović,Bogdan,F,,38.000000:34,...,0.0,10.0,10.0,4.0,0.0,0.0,5.0,1.0,30.0,19.0
1,0022400948,1610612746,LAC,LA,201587,Nicolas Batum,Nicolas,F,,32.000000:52,...,0.0,2.0,2.0,2.0,1.0,0.0,1.0,1.0,6.0,4.0
2,0022400948,1610612746,LAC,LA,1627826,Ivica Zubac,Ivica,C,,38.000000:00,...,6.0,8.0,14.0,1.0,0.0,0.0,2.0,2.0,26.0,5.0
3,0022400948,1610612746,LAC,LA,1627739,Kris Dunn,Kris,G,,23.000000:50,...,1.0,1.0,2.0,3.0,3.0,0.0,1.0,2.0,4.0,4.0
4,0022400948,1610612746,LAC,LA,201935,James Harden,James,G,,38.000000:31,...,3.0,5.0,8.0,11.0,2.0,0.0,3.0,1.0,24.0,15.0
5,0022400948,1610612746,LAC,LA,1627884,Derrick Jones Jr.,Derrick,,,23.000000:26,...,0.0,4.0,4.0,0.0,1.0,0.0,2.0,0.0,8.0,5.0
6,0022400948,1610612746,LAC,LA,1629599,Amir Coffey,Amir,,,25.000000:16,...,0.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,13.0,5.0
7,0022400948,1610612746,LAC,LA,1629234,Drew Eubanks,Drew,,,9.000000:26,...,0.0,2.0,2.0,1.0,0.0,0.0,1.0,2.0,6.0,8.0
8,0022400948,1610612746,LAC,LA,201988,Patty Mills,Patty,,,2.000000:17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
9,0022400948,1610612746,LAC,LA,1641738,Kobe Brown,Kobe,,,0.000000:34,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0


In [ ]:



# Get a list of all NBA players
nba_players = players.get_players()

# Initialize a list to store player seasons
player_seasons = []


# Iterate through the list of players
for player in nba_players:
    player_id = player['id']
    player_name = player['full_name']

    print(f"Processing {player_name}...")  # Print the player name

    # Retrieve player career statistics
    career_stats = playercareerstats.PlayerCareerStats(player_id=player_id)

    # Get the DataFrame of player career stats
    career_stats_df = career_stats.get_data_frames()[0]
